In [ ]:
# default_exp atmosphere

# Atmosphere
> 

4. Dark current subtraction
5. Flat field correction to get radiance from digital number
6. Radiance to Reflectance

In [1]:
#hide

# documentation extraction for class methods
from nbdev.showdoc import *

# unit tests using test_eq(...)
from fastcore.test import *

# monkey patching class methods using @patch
from fastcore.foundation import *
from fastcore.foundation import patch

# imitation of Julia's multiple dispatch using @typedispatch
from fastcore.dispatch import typedispatch


In [2]:
from openhsi.utils import *
from openhsi.calibrate import *
from openhsi.capture import *

/Users/eway/.pyenv/versions/3.8.3/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
#export
from fastcore.foundation import patch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import datetime

from ximea import xiapi
from tqdm import tqdm
import json
from typing import Iterable, Union

In [7]:
openhsi_settings = settings_load("calibration_files/settings.txt")
openhsi_settings["fwhm_nm"]

1.5

In [ ]:
"""
processed_data = DataCube(openhsi_settings["resolution"],openhsi_settings["row_slice"],
                          n_lines=n,axis=1,shifts=openhsi_settings["smile_shifts"],
                          wavelength_range=openhsi_settings["index2wavelength_range"][:2],
                          fwhm_nm=openhsi_settings["fwhm_nm"])


# OpenHSI object will init and read settings file automatically so no need to do above

with OpenHSI(xbinwidth=896,xbinoffset=528,exposure_ms=100,gain=0) as cam:
        img = cam.get_img(n,show=False)
        #processed_data.push(img) # DataCube and OpenHSI classes not integrated together yet. Can push n times though
"""

In [ ]:
luminance_range = np.arange(0,10_000,1000)
exposure_range  = np.array([10,15,20])
n = 10

with OpenHSI(exposure_ms=10,gain=0) as cam:
    hcube = cam.get_datacube(n=1,show=False)
    shape = np.shape(hcube)

for lum in luminance_range:
    
    input(f"Press the Enter key when you are ready to start collecting frames at luminance = {lum} cd/m^2")
    
    for exposure in exposure_range:
        with OpenHSI(exposure_ms=exposure,gain=0) as cam:
            hcube = cam.get_datacube(n=n,show=False)
            averaged = hcube.sum(axis=-1)

with OpenHSI(exposure_ms=10,gain=0) as cam:
    hcube = cam.get_datacube(n=100)
    averaged = hcube.sum(axis=-1)
    
 = CircArrayBuffer(size=np.shape(averaged),dtype=np.uint32)

def __init__(self,size:tuple=(100,100),axis:int=0,dtype=np.uint16,show_func=None):

In [ ]:
radiance_ref = xr.Dataset(data_vars=dict(datacube=(["x","y","wavelength"],processed_data.data)),
                    coords=dict(
                        x=(["x"],np.arange(processed_data.size[0])),
                        y=(["y"],np.arange(processed_data.size[1])),
                        wavelength=(["wavelength"],wavelengths),
                    ),
                    attrs=openhsi_settings["metadata"])